# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *



In [ ]:
#| export
import os
import tempfile
import networkx as nx
import tree_sitter_python as tspython

from glob import glob
from difflib import get_close_matches
from repo2graph.utils import clone_repo
from tree_sitter import Language, Parser

def add_node_by_path(path: str, base_dir: str, repo_name:str, g: nx.DiGraph = None):
    """Adds nodes to the graph based on the file path and adds source code for .py files."""
    if g is None:
        g = nx.DiGraph()

    # Remove the base_dir from the path using removeprefix (Python 3.9+)
    path = path.removeprefix(base_dir + '/')

    # Split the remaining path into components
    components = [repo_name] + path.split('/')

    # Traverse the components and add nodes
    current_node = None  # The root node is not predefined

    for i in range(len(components)):
        next_node = '/'.join(components[:i+1])
        g.add_node(next_node)
        if current_node:
            g.add_edge(current_node, next_node)
        current_node = next_node

    if current_node.endswith('.py'):
        try:
            with open(os.path.join(base_dir, path), 'r') as file:
                source_code = file.read()
            # Add the source code as an attribute to the final node
            g.nodes[current_node]['source'] = source_code
            g = parse_py(g, current_node)
        except Exception as e:
            print(f"Error reading file {path}: {e}")

    return g
        
def add_node_by_repo(repo_url: str = None, repo_dir: str = None, g: nx.DiGraph = None, ignore_files=['setup.py']):
    """Clones the repository (if repo_url is provided) or uses the local directory (if repo_dir is provided),
    and adds nodes based on the files in the repo."""

    # Ensure that either repo_url or repo_dir is provided, but not both
    if bool(repo_url) == bool(repo_dir):
        raise ValueError("You must provide exactly one of 'repo_url' or 'repo_dir', not both.")

    # If no graph is provided, initialize a new one
    if g is None:
        g = nx.DiGraph()

    # Determine the repo name based on the URL or directory
    if repo_url:
        repo_name = repo_url.split('/')[-1].replace('.git', '')
    else:
        repo_name = os.path.basename(os.path.normpath(repo_dir))

    # Use the local directory if provided, otherwise clone the repo
    if repo_url:
        with tempfile.TemporaryDirectory() as tempdir:
            # Clone the repository into the temporary directory
            clone_repo(repo_url, tempdir)

            # Get all file paths in the repository (recursively)
            repo_file_paths = glob(f"{tempdir}/**/*.py", recursive=True)
            # Add nodes to the graph for each file path
            for file_path in repo_file_paths:
                if file_path.split('/')[-1] in ignore_files:
                    continue
                # Add the node based on the file path, using base_dir as the root
                g = add_node_by_path(file_path, repo_dir or tempdir, repo_name, g)
    else:
        repo_file_paths = glob(f"{repo_dir}/**/*.py", recursive=True)

        # Add nodes to the graph for each file path
        for file_path in repo_file_paths:
            if file_path.split('/')[-1] in ignore_files:
                continue
            # Add the node based on the file path, using base_dir as the root
            g = add_node_by_path(file_path, repo_dir or tempdir, repo_name, g)

    return g

def parse_py(g: nx.DiGraph, root_name: str):
    """
    Extract class, function, and import relationships from a syntax tree into a given graph.

    Args:
        g (nx.DiGraph): A pre-existing directed graph.
        root_name (str): The root name of the syntax tree, which contains the `source` attribute.
    """
    PY_LANGUAGE = Language(tspython.language())
    parser = Parser(PY_LANGUAGE)
    tree = parser.parse(bytes(g.nodes[root_name]['source'], "utf8"))
    
    def get_source_text(node):
        """Extract the source code corresponding to a node."""
        return bytes(g.nodes[root_name]['source'], "utf8")[node.start_byte:node.end_byte].decode("utf8")
    
    def traverse(node, current_parent):
        if node.type in ['class_definition', 'function_definition']:
            name_node = node.child_by_field_name('name')
            if name_node:
                name = bytes(g.nodes[root_name]['source'], "utf8")[name_node.start_byte:name_node.end_byte].decode("utf8")
                full_name = '/'.join([current_parent, name])  # Construct hierarchical name
                source_text = get_source_text(node)
                node_type = 'class' if node.type == 'class_definition' else 'function'
                
                # Get start and end lines for function definitions
                start_line = node.start_point[0] + 1  # Adding 1 because line numbers are 1-based
                end_line = node.end_point[0] + 1
                
                # Add node with attributes
                node_attributes = {
                    'type': node_type,
                    'parent': current_parent,
                    'source': source_text,
                }
                
                if node_type == 'function':  # Only functions get start_line and end_line
                    node_attributes['start_line'] = start_line
                    node_attributes['end_line'] = end_line
                
                g.add_node(full_name, **node_attributes)
                
                # Connect to the parent if present
                if current_parent:
                    g.add_edge(current_parent, full_name)

                # If it's a class, traverse its body to extract methods
                if node_type == 'class':
                    body_node = node.child_by_field_name('body')
                    if body_node:  # Recursively traverse the class body
                        traverse(body_node, full_name)

            # Set current_parent to full_name for child functions or classes
            current_parent = full_name if node.type == 'class_definition' else current_parent
        
        # Recursively traverse other children
        for child in node.children:
            traverse(child, current_parent)
    
    traverse(tree.root_node, root_name)
    return g

def search_node_name_by_name(g: nx.DiGraph, query: str, cutoff=0.6):
    """
    Searches for nodes in the graph whose names are closest to the query string.

    Parameters:
    - g: A directed graph (nx.DiGraph).
    - query: The query string to search for.
    - n: The maximum number of close matches to return (default is 3).
    - cutoff: The similarity threshold (default is 0.6, range 0 to 1).

    Returns:
    - A list of nodes whose names are close to the query string.
    """
    # Get all node names (assuming the node names are strings)
    node_names = list(g.nodes)
    py_node_names = [node_name for node_name in node_names if node_name.endswith('.py')]
    
    # Find the closest matches to the query using difflib.get_close_matches
    match = get_close_matches(query, py_node_names, n=1, cutoff=cutoff)
    if not match:
        return None
    else:
        return match[0]
    
def search_func_name_by_name_and_line(g: nx.DiGraph, query: str, line:int, cutoff=0.6):
    py_node_name = search_node_name_by_name(g, query)
    if not py_node_name:
        return None
    for child_node_name in list(g[py_node_name]):
        child_node_attr = g.nodes[child_node_name]
        if child_node_attr.get('type')=='class':
            for child_child_node_name in list(g[child_node_name]):
                child_child_node_attr = g.nodes[child_child_node_name]
                if child_child_node_attr.get('start_line',float('inf')) <= line <= child_child_node_attr.get('end_line',float('-inf')):
                    return child_child_node_name
        else:
            if child_node_attr.get('start_line',float('inf')) <= line <= child_node_attr.get('end_line',float('-inf')):
                return child_node_name
    return None
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()